# Chatbot for Q&A

### This chatbot can answer questions from {https://rbidocs.rbi.org.in/rdocs/notification/PDFs/106MDNBFCS1910202343073E3EF57A4916AA5042911CD8D562.PDF}

In [22]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [23]:
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings

In [24]:
# Embedding
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# LLM
llm = ChatGroq(
    groq_api_key = groq_api_key,
    model = "llama-3.1-8b-instant"      # or "openai/gpt-oss-120b"
)

In [25]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("RBI.pdf")
documents = loader.load()

In [26]:
documents

[Document(metadata={'producer': 'Adobe PDF Library 25.1.213', 'creator': 'Acrobat PDFMaker 25 for Word', 'creationdate': '2025-09-22T12:30:47+05:30', 'comments': '', 'company': 'home', 'keywords': '', 'moddate': '2025-09-22T12:37:19+05:30', 'sourcemodified': 'D:20250922065719', 'subject': '', 'title': '', 'source': 'RBI.pdf', 'total_pages': 330, 'page': 0, 'page_label': '1'}, page_content='RESERVE BANK OF INDIA \nDEPARTMENT OF REGULATION  \nCENTRAL OFFICE, 2ND FLOOR, MAIN OFFICE BUILDING \nSHAHID BHAGAT SINGH MARG, FORT, MUMBAI – 400 001 \n  \nRBI/DoR/2023-24/106 \nDoR.FIN.REC.No.45/03.10.119/2023-24                                           October 19, 2023 \n                                                                                       (Updated as on July 17, 2025)  \n                                                                                       (Updated as on May 05, 2025) \n                                                                               (Updated as on

In [27]:
# Splitting and create Embeddings for documents
from langchain.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size= 2500, chunk_overlap= 300)
splits = text_splitter.split_documents(documents= documents)
vectorStore = FAISS.from_documents(documents= splits, embedding= embeddings)
# Retriever with more results
retriever = vectorStore.as_retriever(search_kwargs={"k": 8})

In [28]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [29]:
# Contextualized Prompt
contextualize_que_system_prompt = (
    "You are a query reformulation assistant for an RBI chatbot. "
    "Your task is to take the most recent user query and rewrite it as a clear, "
    "standalone question in the official FAQ style of the Reserve Bank of India.\n\n"
    
    "Guidelines:\n"
    "- Always use a formal, regulatory tone, similar to RBI FAQs.\n"
    "- If the user query is vague or informal, rewrite it into a precise and self-contained "
    "RBI-style FAQ question.\n"
    "- Explicitly include RBI context terms such as 'NBFCs', 'public deposits', "
    "'Certificate of Registration','guidelines' if they are implied but missing.\n"
    "- Ensure the reformulated question stands alone without needing prior chat history.\n"
    "- Do not answer the question. Only output the reformulated FAQ-style question.\n\n"
    
    "Output Example:\n"
    "User asks: 'can nbfc take deposits?'\n"
    "You reformulate as: 'Can all NBFCs accept deposits? What conditions are prescribed by the Reserve Bank of India for acceptance of public deposits?'"
)



contextualize_que_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_que_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}")
        ]
    )

# History aware retriever
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_que_prompt
)

In [30]:
# QnA Prompt
system_prompt = (
    "You are an AI assistant specializing in Reserve Bank of India (RBI) regulations, "
    "FAQs, circulars, and guidelines. Answer user questions using the retrieved RBI context "
    "as the primary source.\n\n"

    "Answering Style:\n"
    "- Respond in the official FAQ tone of the RBI.\n"
    "- Use short bullet points where helpful.\n"
    "- Include references to specific provisions, "
    "credit rating requirements, deposit limits, or timelines, when available.\n"
    "- Keep responses formal, regulatory, and precise.\n\n"

    "Rules:\n"
    "1. Always try to answer using the retrieved RBI context first.\n"
    "2. If the retrieved context does not contain the information, then use your broader internet knowledge "
    "to provide an authoritative RBI-style answer.\n"
    "3. If neither the context nor broader knowledge provides the answer, reply exactly: "
    "'I could not find this information in the RBI documents or online sources.'\n"
    "4. Never copy text verbatim; paraphrase in clear RBI language.\n"
    "5. Do not mention 'retrieved context' or 'documents'.\n"
    "6. Always keep your response concise, do not give too large response.\n"
    "7. Never include filler phrases such as 'based on my knowledge'—keep the official FAQ tone.\n\n"

    "Retrieved RBI Context:\n"
    "{context}"
)

In [31]:
qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}")
        ]
    )

In [32]:
# Create Chain
qna_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, qna_chain)

In [33]:
res = rag_chain.invoke({"input": "What action can be taken against persons/financial companies making false claim of being regulated by the Reserve Bank?"})

res["answer"]

'According to RBI guidelines, NBFCs shall not claim to be "regulated" by the Reserve Bank of India, if they do not possess a valid Certificate of Registration from the Reserve Bank.\n\nThe Reserve Bank may take penal action under the provisions of the RBI Act, 1934, against persons/financial companies making false claims of being regulated by the Reserve Bank.\n\nThe RBI may also take action under the following sections:\n\n- Section 52 of the Reserve Bank of India Act, 1934: Punishment for false statement.\n- The Consumer Protection Act, 1986: Protection of consumers from unfair trade practices.\n- The Information Technology Act, 2000: Prohibition of misuse of computer resource.\n\nThe Reserve Bank may also order the concerned NBFC to cease and desist from making such false claims and may also initiate enforcement actions under the RBI Act, 1934.\n\n(Related to RBI Act, 1934, and related circulars/guidelines issued by the Reserve Bank.)'

In [34]:
res = rag_chain.invoke({"input": "Why is the Reserve Bank so restrictive in allowing NBFCs to raise public deposits?"})

res["answer"]

"The Reserve Bank of India (RBI) is restrictive in allowing Non-Banking Financial Companies (NBFCs) to raise public deposits, primarily due to concerns about their safety and soundness, as well as the need to protect depositors' interests.\n\nAs per the RBI guidelines, NBFCs are not permitted to accept public deposits, except in certain circumstances. This is because NBFCs are not regulated by the Deposit Insurance and Credit Guarantee Corporation (DICGC), which provides a safety net for bank deposits. The RBI wants to ensure that depositors' funds are protected and that NBFCs do not take on excessive risk.\n\nThe RBI's restrictions on NBFCs raising public deposits are aimed at ensuring that they maintain a stable financial position, adhere to strict prudential norms, and do not indulge in reckless lending or investment practices. By limiting NBFCs' access to public deposits, the RBI seeks to prevent potential systemic risks and maintain financial stability.\n\nReferences: \n- Section 

# Evaluations for this chatbot

### correct questions and answers {https://www.rbi.org.in/commonman/english/scripts/FAQs.aspx?Id=1167}

In [35]:
from langchain.evaluation.criteria.eval_chain import LabeledCriteriaEvalChain


# Create labeled evaluator
evaluator = LabeledCriteriaEvalChain.from_llm(
    llm=llm,
    criteria="correctness"  # we want to check correctness against reference
)


# Ask the chatbot a question, compare its answer with the correct answer and return evaluation results
def evaluate_bot(question: str, correct_answer: str, chain):    
    # Get bot response
    response = chain.invoke({"input": question})
    bot_answer = response["answer"]

    # Evaluate
    eval_result = evaluator.evaluate_strings(
        input=question,
        prediction=bot_answer,
        reference=correct_answer
    )

    return {
        "question": question,
        "expected": correct_answer,
        "predicted": bot_answer,
        "evaluation": eval_result
    }

In [36]:
# Que no. 40
result = evaluate_bot(
    question="Whether NBFCs can accept deposits from NRIs?",
    correct_answer="NBFCs can accept deposits from NRIs subject to compliance with Foreign Exchange Management (Deposit) Regulations 2016 (as amended from time to time) and also subject to the condition that the rate of interest on these deposits shall not exceed the rate specified by the Reserve Bank for such deposits with scheduled commercial banks.",
    chain=rag_chain
)

print(f"Question Asked:\n{result["question"]}")
print()
print(f"Correct Answer:\n{result["expected"]}")
print()
print(f"Bot's Response:\n{result["predicted"]}")

print()
print(f"Evaluation:\n\n Reasoning:\n{result["evaluation"]["reasoning"]}\n\n Score(Pass if 1 Fail if 0):{result["evaluation"]["score"]}")


Question Asked:
Whether NBFCs can accept deposits from NRIs?

Correct Answer:
NBFCs can accept deposits from NRIs subject to compliance with Foreign Exchange Management (Deposit) Regulations 2016 (as amended from time to time) and also subject to the condition that the rate of interest on these deposits shall not exceed the rate specified by the Reserve Bank for such deposits with scheduled commercial banks.

Bot's Response:
Acceptance of Deposits from NRIs by NBFCs  
Non-Deposit Accepting NBFCs are prohibited from accepting deposits from Non-Resident Indians (NRIs). Non-Deposit Accepting NBFCs may, however, act as sub-agents under Money Transfer Service Schemes (MTSS) without any prior approval of the Reserve Bank.

Evaluation:

 Reasoning:
To assess whether the submission meets the criteria, I will break down the reasoning step by step.

1. **Correctness**: 
- The submission states that Non-Deposit Accepting NBFCs are prohibited from accepting deposits from NRIs. 
- However, the refe

In [37]:
# Que no. 51
result = evaluate_bot(
    question="Does the Reserve Bank guarantee the repayment of the deposits collected by NBFCs?",
    correct_answer="No. The Reserve Bank does not guarantee repayment of deposits by NBFCs even though they may be authorized to collect deposits. As such, depositors should take informed decisions while placing deposits with an NBFC.",
    chain=rag_chain
)

print(f"Question Asked:\n{result["question"]}")
print()
print(f"Correct Answer:\n{result["expected"]}")
print()
print(f"Bot's Response:\n{result["predicted"]}")

print()
print(f"Evaluation:\n\n Reasoning:\n{result["evaluation"]["reasoning"]}\n\n Score(Pass if 1 Fail if 0):{result["evaluation"]["score"]}")


Question Asked:
Does the Reserve Bank guarantee the repayment of the deposits collected by NBFCs?

Correct Answer:
No. The Reserve Bank does not guarantee repayment of deposits by NBFCs even though they may be authorized to collect deposits. As such, depositors should take informed decisions while placing deposits with an NBFC.

Bot's Response:
No, the Reserve Bank of India (RBI) does not guarantee the repayment of the deposits collected by Non-Banking Financial Companies (NBFCs). 

As per RBI guidelines, NBFCs are not deposit-taking institutions and do not accept deposits, except for certain permitted exceptions. The RBI Act, 1934, specifically excludes NBFCs from the definition of 'banking company' under Section 5(b) and Section 45-I(c). 

Investments in fixed deposits, as mentioned in provision 74, shall not be treated as financial assets, and receipt of interest income on fixed deposits with banks shall not be treated as income from financial assets. This further confirms that NBFC

In [38]:
# Que no. 79
result = evaluate_bot(
    question="What precautions have to be taken by the public to forewarn themselves about the likelihood of losing money in schemes that offer high rates of interest?",
    correct_answer="Before investing in schemes that promise high rates of return, the depositors/ investors must ensure that the entity offering such returns is registered with one of the Financial Sector Regulators and is authorized to accept funds, whether in the form of deposits or otherwise. Depositors/ investors must generally be circumspect if the interest rates or rates of return on deposits/ investments offered are high. Unless the entity accepting funds is able to earn more than what it promises, the entity will not be able to repay the depositor/ investor as promised. For earning higher returns, the entity will have to take higher risks on the investments it makes. Higher the risk, the more speculative would be its investments on which there can be no assured return. As such, members of public should forewarn themselves that the likelihood of losing money in schemes that offer high rates of interest are more.",
    chain=rag_chain
)

print(f"Question Asked:\n{result["question"]}")
print()
print(f"Correct Answer:\n{result["expected"]}")
print()
print(f"Bot's Response:\n{result["predicted"]}")

print()
print(f"Evaluation:\n\n Reasoning:\n{result["evaluation"]["reasoning"]}\n\n Score(Pass if 1 Fail if 0):{result["evaluation"]["score"]}")


Question Asked:
What precautions have to be taken by the public to forewarn themselves about the likelihood of losing money in schemes that offer high rates of interest?

Correct Answer:
Before investing in schemes that promise high rates of return, the depositors/ investors must ensure that the entity offering such returns is registered with one of the Financial Sector Regulators and is authorized to accept funds, whether in the form of deposits or otherwise. Depositors/ investors must generally be circumspect if the interest rates or rates of return on deposits/ investments offered are high. Unless the entity accepting funds is able to earn more than what it promises, the entity will not be able to repay the depositor/ investor as promised. For earning higher returns, the entity will have to take higher risks on the investments it makes. Higher the risk, the more speculative would be its investments on which there can be no assured return. As such, members of public should forewarn t

In [39]:
# Que no. 81
result = evaluate_bot(
    question="What constitutes Commercial Real Estate exposure?",
    correct_answer="Commercial Real Estate (CRE) would consist of loans to builders/ developers/ others for creation/ acquisition of commercial real estate (such as office building, retail space, multi-purpose commercial premises, multi-tenanted commercial premises, industrial or warehouse space, hotels, land acquisition, development and construction etc.) where the prospects for repayment, or recovery in case of default, would depend primarily on the cash flows generated by the asset by way of lease/rental payments, sale etc. Further, loans for third dwelling unit onwards to an individual will be treated as CRE exposure. Exposure shall also include non-fund based limits. Commercial Real Estate – Residential Housing (CRE–RH) is a sub-category of CRE that consist of loans to builders/ developers for residential housing projects (except for captive consumption). Such projects should ordinarily not include non-residential commercial real estate. However integrated housing project comprising of some commercial spaces (e.g., shopping complex, school etc.) can also be specified under CRE-RH, provided that the commercial area in the residential housing project does not exceed 10 percent of the total Floor Space Index (FSI) of the project. In case the FSI of the commercial area in the predominantly residential housing complex exceed the ceiling of 10 percent, the entire loan should be classified as CRE and not CRE-RH.",
    chain=rag_chain
)

print(f"Question Asked:\n{result["question"]}")
print()
print(f"Correct Answer:\n{result["expected"]}")
print()
print(f"Bot's Response:\n{result["predicted"]}")

print()
print(f"Evaluation:\n\n Reasoning:\n{result["evaluation"]["reasoning"]}\n\n Score(Pass if 1 Fail if 0):{result["evaluation"]["score"]}")


Question Asked:
What constitutes Commercial Real Estate exposure?

Correct Answer:
Commercial Real Estate (CRE) would consist of loans to builders/ developers/ others for creation/ acquisition of commercial real estate (such as office building, retail space, multi-purpose commercial premises, multi-tenanted commercial premises, industrial or warehouse space, hotels, land acquisition, development and construction etc.) where the prospects for repayment, or recovery in case of default, would depend primarily on the cash flows generated by the asset by way of lease/rental payments, sale etc. Further, loans for third dwelling unit onwards to an individual will be treated as CRE exposure. Exposure shall also include non-fund based limits. Commercial Real Estate – Residential Housing (CRE–RH) is a sub-category of CRE that consist of loans to builders/ developers for residential housing projects (except for captive consumption). Such projects should ordinarily not include non-residential comm

In [40]:
# Que no. 79
result = evaluate_bot(
    question="What does conducting financial activity as “principal business” mean?",
    correct_answer="Financial activity as principal business is when a company’s financial assets constitute more than 50 per cent of the total assets (netted off by intangible assets) and income from financial assets constitute more than 50 per cent of the gross income. A company which fulfils both these criteria needs to get registered as NBFC with the Reserve Bank. The term 'principal business' has not been defined in the Reserve Bank of India Act, 1934. Hence, the Reserve Bank has defined it vide Press Release 1998-99/1269 dated April 08, 1999 so as to ensure that only companies predominantly engaged in financial activity get registered with it and are regulated and supervised by it. Hence, if there are companies engaged in agricultural operations, industrial activity, purchase and sale of goods, providing services or purchase, sale or construction of immovable property as their principal business and are doing some financial business in a small way, they will not be regulated by the Reserve Bank. Interestingly, this test is popularly known as 50-50 test and is applied to determine whether or not a company is into financial business.",
    chain=rag_chain
)

print(f"Question Asked:\n{result["question"]}")
print()
print(f"Correct Answer:\n{result["expected"]}")
print()
print(f"Bot's Response:\n{result["predicted"]}")

print()
print(f"Evaluation:\n\n Reasoning:\n{result["evaluation"]["reasoning"]}\n\n Score(Pass if 1 Fail if 0):{result["evaluation"]["score"]}")


Question Asked:
What does conducting financial activity as “principal business” mean?

Correct Answer:
Financial activity as principal business is when a company’s financial assets constitute more than 50 per cent of the total assets (netted off by intangible assets) and income from financial assets constitute more than 50 per cent of the gross income. A company which fulfils both these criteria needs to get registered as NBFC with the Reserve Bank. The term 'principal business' has not been defined in the Reserve Bank of India Act, 1934. Hence, the Reserve Bank has defined it vide Press Release 1998-99/1269 dated April 08, 1999 so as to ensure that only companies predominantly engaged in financial activity get registered with it and are regulated and supervised by it. Hence, if there are companies engaged in agricultural operations, industrial activity, purchase and sale of goods, providing services or purchase, sale or construction of immovable property as their principal business an